In [9]:
from scipy.stats import norm

eta_conf = 95/100       # -
P_1_12_fore = 100       # kW
sigma_1_12_fore = 5     # kw

P_1_12 = P_1_12_fore + sigma_1_12_fore*norm.ppf(1 - eta_conf)
print(P_1_12)
print(norm.ppf(1 - eta_conf, loc=P_1_12_fore, scale=sigma_1_12_fore))

91.77573186524263
91.77573186524263


The chance-constrained optimization approach basically limits the RES output to a value smaller than maximum expected output. Since they are trying to maximize profits, the optimization problem will always push this constraint to the upper limit.